In [1]:
# Imports 
import numpy as np 
import pandas as pd 
import xlsxwriter
import requests 
from scipy import stats 
import math 

In [2]:
# Importing our list of stocks 
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
# Acquiring API token 
from secrets import IEX_CLOUD_API_TOKEN

In [5]:
# Batch API Call Function 
def chunks(lst, n): 
    for i in range(0, len(lst), n): 
        yield lst[i:i + n] 

columns = ['Ticker', 'Stock Price', 'Price to Earnings Ratio', 'Number of Shares to Buy']

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)): 
    symbol_strings.append(','.join(symbol_groups[i]))

In [6]:
final_dataframe = pd.DataFrame(columns=columns)

for symbol_string in symbol_strings: 
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'                
                ], 
                index = columns
            ),
            ignore_index=True
        )
final_dataframe

,Ticker,Stock Price,Price to Earnings Ratio,Number of Shares to Buy
0,A,126.58,54.08,N/A
1,AAL,17.21,-1.28,N/A
2,AAP,159.82,23.06,N/A
3,AAPL,140.16,38.6,N/A
4,ABBV,111.56,24.64,N/A
...,...,...,...,...
500,YUM,110.51,30.65,N/A
501,ZBH,164.45,-218.24,N/A
502,ZBRA,407.80,48.38,N/A
503,ZION,49.74,21.31,N/A


In [7]:
# Removing Stocks 
final_dataframe.sort_values('Price to Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price to Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)

In [8]:
final_dataframe = final_dataframe.drop(['index'], axis = 1)
final_dataframe.head()

,Ticker,Stock Price,Price to Earnings Ratio,Number of Shares to Buy
0,NRG,42.86,2.58,N/A
1,NLOK,21.55,4.23,N/A
2,AIV,4.93,4.79,N/A
3,UNM,23.53,5,N/A
4,BIO,618.50,5.4,N/A


In [9]:
def portfolio_input(): 
    global portfolio_size
    portfolio_size = input('Enter the Size of your Portfolio')

    try: 
        float(portfolio_size)
    except ValueError: 
        print('Please enter an integer')
        portfolio_size = input('Enter the Value of Portfolio: ')
        portfolio_size = float(portfolio_size)

In [10]:
portfolio_input()

In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)


for i in range(0, len(final_dataframe.index)): 
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Price to Earnings Ratio,Number of Shares to Buy
0,NRG,42.86,2.58,466
1,NLOK,21.55,4.23,928
2,AIV,4.93,4.79,4056
3,UNM,23.53,5,849
4,BIO,618.50,5.4,32
5,AFL,46.76,7.33,427
6,ALL,111.52,7.51,179
7,CPB,49.00,8.23,408
8,KIM,16.93,8.31,1181
9,EBAY,63.61,8.79,314


In [55]:
# Composite Strategy
rv_columns = [
    'Ticker', 
    'Stock Price',
    'Number of Shares to Buy', 
    'Price to Earnings Ratio',
    'PE Percentile',
    'Price to Book Ratio', 
    'PB Percentile', 
    'Price to Sales Ratio', 
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile', 
    'EV/GP',
    'EV/GP Percentile', 
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

In [26]:
len(rv_columns)

13

In [56]:
# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [57]:
# Building Dataframe
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'                
            ], 
            index = rv_columns
            ),
            ignore_index = True
        )
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.79,N/A,54.53,N/A,8.06,N/A,7.38,N/A,33.193373,N/A,13.611019,N/A,N/A
1,AAL,17.78,N/A,-1.29,N/A,-1.96,N/A,0.4421,N/A,-6.211660,N/A,1.443711,N/A,N/A
2,AAP,158.84,N/A,22.9,N/A,2.96,N/A,1.09,N/A,10.686668,N/A,2.475126,N/A,N/A
3,AAPL,142.14,N/A,38.57,N/A,36,N/A,7.97,N/A,26.855773,N/A,19.669115,N/A,N/A
4,ABBV,110.24,N/A,24.74,N/A,12.99,N/A,4.9,N/A,16.111137,N/A,9.622763,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.05,N/A,31.56,N/A,-4.12,N/A,5.72,N/A,22.500648,N/A,9.832610,N/A,N/A
501,ZBH,161.36,N/A,-218.31,N/A,2.84,N/A,4.83,N/A,26.439562,N/A,7.795687,N/A,N/A
502,ZBRA,417.58,N/A,47.29,N/A,11.35,N/A,5.02,N/A,31.800408,N/A,12.327672,N/A,N/A
503,ZION,49.54,N/A,20.98,N/A,1.1,N/A,2.67,N/A,11.770931,N/A,3.053665,N/A,N/A


In [58]:
# Checking for Nulls 
len(rv_dataframe[rv_dataframe.isnull().any(axis=1)])

17

In [59]:
for col in ['Price to Earnings Ratio', 'Price to Book Ratio', 'Price to Sales Ratio','Price to Sales Ratio', 'EV/EBITDA', 'EV/GP']: 
    rv_dataframe[col].fillna(rv_dataframe[col].mean(), inplace=True)

In [60]:
len(rv_dataframe[rv_dataframe.isnull().any(axis=1)])

0

In [61]:
# Calculating RV Percentile 
metrics = {
  'Price to Earnings Ratio':'PE Percentile',
    'Price to Book Ratio':'PB Percentile', 
    'Price to Sales Ratio': 'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile', 
    'EV/GP':'EV/GP Percentile', 
}

for metric in metrics.keys(): 
    for row in rv_dataframe.index: 
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.79,N/A,54.53,0.865347,8.06,0.744554,7.3800,0.809901,33.193373,0.847525,13.611019,0.80198,N/A
1,AAL,17.78,N/A,-1.29,0.152475,-1.96,0.049505,0.4421,0.0316832,-6.211660,0.029703,1.443711,0.0554455,N/A
2,AAP,158.84,N/A,22.90,0.465347,2.96,0.39802,1.0900,0.132673,10.686668,0.213861,2.475126,0.108911,N/A
3,AAPL,142.14,N/A,38.57,0.754455,36.00,0.962376,7.9700,0.831683,26.855773,0.780198,19.669115,0.934653,N/A
4,ABBV,110.24,N/A,24.74,0.491089,12.99,0.856436,4.9000,0.69604,16.111137,0.477228,9.622763,0.637624,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.05,N/A,31.56,0.649505,-4.12,0.0455446,5.7200,0.742574,22.500648,0.683168,9.832610,0.657426,N/A
501,ZBH,161.36,N/A,-218.31,0.0217822,2.84,0.388119,4.8300,0.689109,26.439562,0.776238,7.795687,0.49505,N/A
502,ZBRA,417.58,N/A,47.29,0.825743,11.35,0.833663,5.0200,0.70099,31.800408,0.839604,12.327672,0.754455,N/A
503,ZION,49.54,N/A,20.98,0.424752,1.10,0.106931,2.6700,0.410891,11.770931,0.283168,3.053665,0.148515,N/A


In [62]:
# Getting RV Score 
from statistics import mean 
for row in rv_dataframe.index: 
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
        rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.79,N/A,54.53,0.865347,8.06,0.744554,7.3800,0.809901,33.193373,0.847525,13.611019,0.80198,0.813861
1,AAL,17.78,N/A,-1.29,0.152475,-1.96,0.049505,0.4421,0.0316832,-6.211660,0.029703,1.443711,0.0554455,0.0637624
2,AAP,158.84,N/A,22.90,0.465347,2.96,0.39802,1.0900,0.132673,10.686668,0.213861,2.475126,0.108911,0.263762
3,AAPL,142.14,N/A,38.57,0.754455,36.00,0.962376,7.9700,0.831683,26.855773,0.780198,19.669115,0.934653,0.852673
4,ABBV,110.24,N/A,24.74,0.491089,12.99,0.856436,4.9000,0.69604,16.111137,0.477228,9.622763,0.637624,0.631683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.05,N/A,31.56,0.649505,-4.12,0.0455446,5.7200,0.742574,22.500648,0.683168,9.832610,0.657426,0.555644
501,ZBH,161.36,N/A,-218.31,0.0217822,2.84,0.388119,4.8300,0.689109,26.439562,0.776238,7.795687,0.49505,0.474059
502,ZBRA,417.58,N/A,47.29,0.825743,11.35,0.833663,5.0200,0.70099,31.800408,0.839604,12.327672,0.754455,0.790891
503,ZION,49.54,N/A,20.98,0.424752,1.10,0.106931,2.6700,0.410891,11.770931,0.283168,3.053665,0.148515,0.274851


In [63]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)


In [64]:
rv_dataframe = rv_dataframe[:50]


In [66]:
rv_dataframe = rv_dataframe.reset_index(drop=True)
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,43.59,N/A,-7.62,0.110891,0.5821,0.0594059,0.7879,0.0732673,4.852489,0.0435644,0.777479,0.0178218,0.0609901
1,UNM,24.42,N/A,5.00,0.194059,0.4367,0.0534653,0.4117,0.029703,2.601669,0.0356436,0.391728,0.0039604,0.0633663
2,AAL,17.78,N/A,-1.29,0.152475,-1.9600,0.049505,0.4421,0.0316832,-6.211660,0.029703,1.443711,0.0554455,0.0637624
3,MCK,185.65,N/A,-7.43,0.112871,-62.3400,0.0118812,0.1233,0.00594059,7.315560,0.0792079,2.716149,0.120792,0.0661386
4,F,11.67,N/A,-294.35,0.019802,1.4300,0.184158,0.3594,0.019802,4.781803,0.0415842,2.451895,0.106931,0.0744554
5,MET,53.75,N/A,8.76,0.205941,0.6455,0.0633663,0.7485,0.0673267,4.901266,0.0455446,0.686810,0.0118812,0.0788119
6,FTI,10.89,N/A,-0.88,0.156436,1.2000,0.127723,0.3789,0.0237624,3.313702,0.0376238,2.146308,0.0990099,0.0889109
7,ALL,108.85,N/A,7.84,0.2,1.3100,0.153465,0.7325,0.0633663,2.497901,0.0336634,0.744939,0.0138614,0.0928713
8,HIG,51.07,N/A,11.01,0.231683,1.0300,0.0980198,0.9082,0.0930693,4.344597,0.039604,0.871929,0.0217822,0.0968317
9,BA,214.80,N/A,-10.36,0.10099,-6.8900,0.0415842,2.1400,0.338614,-14.556827,0.0237624,-27.585347,0.0019802,0.101386


In [67]:
portfolio_input()

In [68]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Stock Price'][i])
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,43.59,1147,-7.62,0.110891,0.5821,0.0594059,0.7879,0.0732673,4.852489,0.0435644,0.777479,0.0178218,0.0609901
1,UNM,24.42,2047,5.00,0.194059,0.4367,0.0534653,0.4117,0.029703,2.601669,0.0356436,0.391728,0.0039604,0.0633663
2,AAL,17.78,2812,-1.29,0.152475,-1.9600,0.049505,0.4421,0.0316832,-6.211660,0.029703,1.443711,0.0554455,0.0637624
3,MCK,185.65,269,-7.43,0.112871,-62.3400,0.0118812,0.1233,0.00594059,7.315560,0.0792079,2.716149,0.120792,0.0661386
4,F,11.67,4284,-294.35,0.019802,1.4300,0.184158,0.3594,0.019802,4.781803,0.0415842,2.451895,0.106931,0.0744554
5,MET,53.75,930,8.76,0.205941,0.6455,0.0633663,0.7485,0.0673267,4.901266,0.0455446,0.686810,0.0118812,0.0788119
6,FTI,10.89,4591,-0.88,0.156436,1.2000,0.127723,0.3789,0.0237624,3.313702,0.0376238,2.146308,0.0990099,0.0889109
7,ALL,108.85,459,7.84,0.2,1.3100,0.153465,0.7325,0.0633663,2.497901,0.0336634,0.744939,0.0138614,0.0928713
8,HIG,51.07,979,11.01,0.231683,1.0300,0.0980198,0.9082,0.0930693,4.344597,0.039604,0.871929,0.0217822,0.0968317
9,BA,214.80,232,-10.36,0.10099,-6.8900,0.0415842,2.1400,0.338614,-14.556827,0.0237624,-27.585347,0.0019802,0.101386


In [69]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [70]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [71]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [72]:
writer.save()